# Import Library

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
# import matplotlib.pyplot as plt

from sklearn import cluster
from sklearn import mixture
from sklearn import preprocessing
from collections import Counter
# from mpl_toolkits.mplot3d import Axes3D

# Data Preprocess
### Reading data

In [ ]:
def read_data(folder):
    file_path = "../data/50mph/" + str(folder) + "/" + str(folder) + "_Step1.csv"
    data = np.abs(pd.read_csv(file_path))
    data["movement_x"] = 0
    data["movement_y"] = 0
    data["movement_z"] = 0

    for i in range(1,303):
        # create file path
        file_path_1 = "../data/50mph/" + str(folder) + "/" + str(folder) + "_Step" + str(i) + ".csv"
        file_path_2 = "../data/50mph/" + str(folder) + "/" + str(folder) + "_Step" + str(i+1) + ".csv"
        print(file_path_1)
        # read in file 
        data_1 = pd.read_csv(file_path_1)
        data_2 = pd.read_csv(file_path_2)

        # calculate movement in x,y,z directions
        data.iloc[:,20:23] = data.iloc[:,20:23] + np.abs(data_2.iloc[:,2:5] - data_1.iloc[:,2:5]).values

        # sum values
        data.iloc[:,5:20] = data.iloc[:,5:20] + np.abs(data_2.iloc[:,5:20])
        
        return data
